# 下載coco資料集
https://cocodataset.org/#download

In [6]:
# from pycocotools.coco import COCO
# import requests
# import os

# # 設定資料集存放路徑
# COCOROOT = 'datasets'
# if not os.path.exists(COCOROOT):
#     os.makedirs(COCOROOT)

# # 設定 COCO 資料集的 URL 和下載目標檔案名稱
# data_types = ['train2017', 'val2017', 'annotations']
# base_url = 'http://images.cocodataset.org/'

# for data_type in data_types:
#     if data_type == 'annotations':
#         url = f'{base_url}annotations/annotations_trainval2017.zip'
#         target_path = os.path.join(COCOROOT, 'annotations_trainval2017.zip')
#     else:
#         url = f'{base_url}zips/{data_type}.zip'
#         target_path = os.path.join(COCOROOT, f'{data_type}.zip')
    
#     response = requests.get(url)
#     with open(target_path, 'wb') as f:
#         f.write(response.content)
#     print(f'{data_type} 資料集下載完成')

# # 下載完成後解壓縮
# import zipfile
# for data_type in data_types:
#     if data_type == 'annotations':
#         zip_path = os.path.join(COCOROOT, 'annotations_trainval2017.zip')
#     else:
#         zip_path = os.path.join(COCOROOT, f'{data_type}.zip')
    
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(COCOROOT)
#     print(f'{data_type} 資料集解壓完成')

# 將ISAT json轉換為mask.png

In [1]:
import os
import json
from PIL import Image, ImageDraw

# Function to process each JSON file
def process_json(json_file, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Get image dimensions from JSON data
    image_width = data['info']['width']
    image_height = data['info']['height']

    # Create a blank mask image (assumed RGB image with white background)
    mask_image = Image.new('RGB', (image_width, image_height), color=(0, 0, 0))
    draw = ImageDraw.Draw(mask_image)

    # Define categories to fill with white color
    categories_to_fill = ["person"]

    # Draw mask for each specified category
    for obj in data['objects']:
        category = obj['category']
        if category in categories_to_fill:
            segmentation = obj['segmentation']
            polygon = []
            for point in segmentation:
                polygon.append(tuple(point))
            draw.polygon(polygon, fill=(255, 255, 255))  # Fill white color

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save the mask image to the specified folder
    json_filename = os.path.splitext(os.path.basename(json_file))[0]
    mask_image.save(os.path.join(output_folder, f'{json_filename}_mask.png'))

    print(f'Mask image for {json_file} has been generated and saved as {json_filename}_mask.png')

# Directory containing JSON files
json_folder = 'datasets/paper_debris/abnormal'
# Output folder for mask images
output_folder = 'datasets/paper_debris/mask'

# Process each JSON file in the directory
for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        json_file = os.path.join(json_folder, filename)
        process_json(json_file, output_folder)


Mask image for datasets/paper_debris/abnormal\2024-06-18 15-11-14.json has been generated and saved as 2024-06-18 15-11-14_mask.png
Mask image for datasets/paper_debris/abnormal\2024-06-18 15-12-02.json has been generated and saved as 2024-06-18 15-12-02_mask.png
Mask image for datasets/paper_debris/abnormal\2024-06-18 15-12-24.json has been generated and saved as 2024-06-18 15-12-24_mask.png
Mask image for datasets/paper_debris/abnormal\2024-06-18 15-12-34.json has been generated and saved as 2024-06-18 15-12-34_mask.png
Mask image for datasets/paper_debris/abnormal\2024-06-18 15-12-56.json has been generated and saved as 2024-06-18 15-12-56_mask.png
Mask image for datasets/paper_debris/abnormal\2024-06-18 15-13-07.json has been generated and saved as 2024-06-18 15-13-07_mask.png
Mask image for datasets/paper_debris/abnormal\2024-06-18 15-13-22.json has been generated and saved as 2024-06-18 15-13-22_mask.png
Mask image for datasets/paper_debris/abnormal\2024-06-18 15-13-32.json has b

# 將標註的帶測物範圍的圖檔轉為去背專注的圖檔格式
1. mask
2. 圖檔基於mask最大範圍擷取
3. 圖檔去背並擷取

In [5]:
# 將標註的帶測物範圍的圖檔轉為去背專注的圖檔格式
import os
import json
from PIL import Image, ImageDraw

# Function to process each JSON file
def process_json(json_file, image_folder, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Get image file name from JSON data
    image_file = data['info']['name']
    image_path = os.path.join(image_folder, image_file)

    # Load the original image
    original_image = Image.open(image_path)
    image_width, image_height = original_image.size

    # Create a blank mask image (assumed RGB image with black background)
    mask_image = Image.new('L', (image_width, image_height), color=0)
    draw = ImageDraw.Draw(mask_image)

    # Create a blank image for the third type (original content in mask area)
    mask_inverted_image = Image.new('RGB', (image_width, image_height), color=(0, 0, 0))

    # Define categories to fill with white color
    categories_to_fill = ["hat"]

    # Variables to track the bounding box of all points
    min_x, min_y = image_width, image_height
    max_x, max_y = 0, 0

    # Draw mask for each specified category
    for obj in data['objects']:
        category = obj['category']
        if category in categories_to_fill:
            segmentation = obj['segmentation']
            polygon = []
            for point in segmentation:
                polygon.append(tuple(point))
                # Update bounding box
                min_x = min(min_x, point[0])
                min_y = min(min_y, point[1])
                max_x = max(max_x, point[0])
                max_y = max(max_y, point[1])
            draw.polygon(polygon, fill=255)  # Fill white color in the mask

    # Create an image with original content in mask area
    mask_image_rgb = mask_image.convert("RGB")
    mask_inverted_image.paste(original_image, mask=mask_image)

    # Crop the original image, mask image, and mask inverted image to the bounding box
    cropped_image = original_image.crop((min_x, min_y, max_x, max_y))
    cropped_mask_image = mask_image.crop((min_x, min_y, max_x, max_y))
    cropped_mask_inverted_image = mask_inverted_image.crop((min_x, min_y, max_x, max_y))

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save the cropped images to the specified folder
    json_filename = os.path.splitext(os.path.basename(json_file))[0]
    cropped_image.save(os.path.join(output_folder, f'{json_filename}_cropped.png'))
    cropped_mask_image.save(os.path.join(output_folder, f'{json_filename}_mask.png'))
    cropped_mask_inverted_image.save(os.path.join(output_folder, f'{json_filename}_mask_inverted.png'))

    print(f'Cropped image, mask, and mask_inverted for {json_file} have been generated and saved as {json_filename}_cropped.png, {json_filename}_mask.png, and {json_filename}_mask_inverted.png')

# Directory containing JSON files
json_folder = 'C:/Users/User/Desktop/normal'
# Directory containing the original images
image_folder = 'C:/Users/User/Desktop/normal'
# Output folder for cropped images and mask images
output_folder = 'datasets/paper_debris/normal_cropped'

# Process each JSON file in the directory
for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        json_file = os.path.join(json_folder, filename)
        process_json(json_file, image_folder, output_folder)


Cropped image, mask, and mask_inverted for C:/Users/User/Desktop/normal\2024-06-18 15-20-02.json have been generated and saved as 2024-06-18 15-20-02_cropped.png, 2024-06-18 15-20-02_mask.png, and 2024-06-18 15-20-02_mask_inverted.png
Cropped image, mask, and mask_inverted for C:/Users/User/Desktop/normal\2024-06-18 15-20-12.json have been generated and saved as 2024-06-18 15-20-12_cropped.png, 2024-06-18 15-20-12_mask.png, and 2024-06-18 15-20-12_mask_inverted.png
Cropped image, mask, and mask_inverted for C:/Users/User/Desktop/normal\2024-06-18 15-20-22.json have been generated and saved as 2024-06-18 15-20-22_cropped.png, 2024-06-18 15-20-22_mask.png, and 2024-06-18 15-20-22_mask_inverted.png
Cropped image, mask, and mask_inverted for C:/Users/User/Desktop/normal\2024-06-18 15-20-48.json have been generated and saved as 2024-06-18 15-20-48_cropped.png, 2024-06-18 15-20-48_mask.png, and 2024-06-18 15-20-48_mask_inverted.png
Cropped image, mask, and mask_inverted for C:/Users/User/Des

# 新增三種檔案

In [ ]:
# 將標註的帶測物範圍的圖檔轉為去背專注的圖檔格式
import os
import json
from PIL import Image, ImageDraw

# Function to process each JSON file
def process_json(json_file, image_folder, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Get image file name from JSON data
    image_file = data['info']['name']
    image_path = os.path.join(image_folder, image_file)

    # Load the original image
    original_image = Image.open(image_path)
    image_width, image_height = original_image.size

    # Create a blank mask image (assumed RGB image with black background)
    mask_image = Image.new('L', (image_width, image_height), color=0)
    draw = ImageDraw.Draw(mask_image)

    # Create a blank image for the third type (original content in mask area)
    mask_inverted_image = Image.new('RGB', (image_width, image_height), color=(0, 0, 0))

    # Define categories to fill with white color
    categories_to_fill = ["hat"]

    # Variables to track the bounding box of all points
    min_x, min_y = image_width, image_height
    max_x, max_y = 0, 0

    # Draw mask for each specified category
    for obj in data['objects']:
        category = obj['category']
        if category in categories_to_fill:
            segmentation = obj['segmentation']
            polygon = []
            for point in segmentation:
                polygon.append(tuple(point))
                # Update bounding box
                min_x = min(min_x, point[0])
                min_y = min(min_y, point[1])
                max_x = max(max_x, point[0])
                max_y = max(max_y, point[1])
            draw.polygon(polygon, fill=255)  # Fill white color in the mask

    # Create an image with original content in mask area
    mask_image_rgb = mask_image.convert("RGB")
    mask_inverted_image.paste(original_image, mask=mask_image)

    # Crop the original image, mask image, and mask inverted image to the bounding box
    cropped_image = original_image.crop((min_x, min_y, max_x, max_y))
    cropped_mask_image = mask_image.crop((min_x, min_y, max_x, max_y))
    cropped_mask_inverted_image = mask_inverted_image.crop((min_x, min_y, max_x, max_y))

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save the cropped images to the specified folder
    json_filename = os.path.splitext(os.path.basename(json_file))[0]
    cropped_image.save(os.path.join(output_folder, f'{json_filename}_cropped.png'))
    cropped_mask_image.save(os.path.join(output_folder, f'{json_filename}_mask.png'))
    cropped_mask_inverted_image.save(os.path.join(output_folder, f'{json_filename}_mask_inverted.png'))

    print(f'Cropped image, mask, and mask_inverted for {json_file} have been generated and saved as {json_filename}_cropped.png, {json_filename}_mask.png, and {json_filename}_mask_inverted.png')

# Directory containing JSON files
json_folder = 'C:/Users/User/Desktop/normal'
# Directory containing the original images
image_folder = 'C:/Users/User/Desktop/normal'
# Output folder for cropped images and mask images
output_folder = 'datasets/paper_debris/normal_cropped'

# Process each JSON file in the directory
for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        json_file = os.path.join(json_folder, filename)
        process_json(json_file, image_folder, output_folder)


In [10]:
import os
import json
from PIL import Image, ImageDraw

# Function to process each JSON file
def process_json(json_file, image_folder, output_folder, anomaly_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Get image file name from JSON data
    image_file = data['info']['name']
    image_path = os.path.join(image_folder, image_file)

    # Load the original image
    original_image = Image.open(image_path)
    image_width, image_height = original_image.size

    # Create a blank mask image (assumed RGB image with black background)
    mask_image = Image.new('L', (image_width, image_height), color=0)
    draw = ImageDraw.Draw(mask_image)

    # Create a blank image for the third type (original content in mask area)
    mask_inverted_image = Image.new('RGB', (image_width, image_height), color=(0, 0, 0))

    # Define categories to fill with white color
    categories_to_fill = ["hat"]

    # Variables to track the bounding box of all points
    min_x, min_y = image_width, image_height
    max_x, max_y = 0, 0

    # Draw mask for each specified category
    for obj in data['objects']:
        category = obj['category']
        if category in categories_to_fill:
            segmentation = obj['segmentation']
            polygon = []
            for point in segmentation:
                polygon.append(tuple(point))
                # Update bounding box
                min_x = min(min_x, point[0])
                min_y = min(min_y, point[1])
                max_x = max(max_x, point[0])
                max_y = max(max_y, point[1])
            draw.polygon(polygon, fill=255)  # Fill white color in the mask

    # Create an image with original content in mask area
    mask_image_rgb = mask_image.convert("RGB")
    mask_inverted_image.paste(original_image, mask=mask_image)

    # Crop the original image, mask image, and mask inverted image to the bounding box
    cropped_image = original_image.crop((min_x, min_y, max_x, max_y))
    cropped_mask_image = mask_image.crop((min_x, min_y, max_x, max_y))
    cropped_mask_inverted_image = mask_inverted_image.crop((min_x, min_y, max_x, max_y))

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save the cropped images to the specified folder
    json_filename = os.path.splitext(os.path.basename(json_file))[0]
    cropped_image.save(os.path.join(output_folder, f'{json_filename}_cropped.png'))
    cropped_mask_image.save(os.path.join(output_folder, f'{json_filename}_mask.png'))
    cropped_mask_inverted_image.save(os.path.join(output_folder, f'{json_filename}_mask_inverted.png'))

    print(f'Cropped image, mask, and mask_inverted for {json_file} have been generated and saved as {json_filename}_cropped.png, {json_filename}_mask.png, and {json_filename}_mask_inverted.png')

    # Process anomaly folder to generate anomaly mask
    anomaly_json_folder = anomaly_folder  # Path to the folder containing anomaly JSON files

    # Process each JSON file in the anomaly folder
    for anomaly_filename in os.listdir(anomaly_json_folder):
        if anomaly_filename.endswith('.json'):
            anomaly_json_file = os.path.join(anomaly_json_folder, anomaly_filename)

            with open(anomaly_json_file, 'r') as af:
                anomaly_data = json.load(af)

            # Create a blank mask image for anomaly mask (assumed RGB image with black background)
            anomaly_mask_image = Image.new('L', (image_width, image_height), color=0)
            anomaly_draw = ImageDraw.Draw(anomaly_mask_image)

            # Draw mask for each specified category in anomaly JSON
            for obj in anomaly_data['objects']:
                category = obj['category']
                if category in categories_to_fill:
                    segmentation = obj['segmentation']
                    polygon = []
                    for point in segmentation:
                        polygon.append(tuple(point))
                    anomaly_draw.polygon(polygon, fill=255)  # Fill white color in the anomaly mask

            # Crop the anomaly mask to the bounding box
            cropped_anomaly_mask_image = anomaly_mask_image.crop((min_x, min_y, max_x, max_y))

            # Save the cropped anomaly mask to the specified folder
            anomaly_json_filename = os.path.splitext(os.path.basename(anomaly_json_file))[0]
            cropped_anomaly_mask_image.save(os.path.join(output_folder, f'{anomaly_json_filename}_anomaly_mask_crop.png'))

            print(f'Anomaly mask cropped for {anomaly_json_file} has been generated and saved as {anomaly_json_filename}_anomaly_mask_crop.png')

# Directory containing JSON files for normal images
json_folder = 'datasets/paper_debris/abnormal_analyte'
# Directory containing JSON files for anomaly images
anomaly_folder = 'datasets/paper_debris/abnormal_anomaly'
# Directory containing the original images
image_folder = 'datasets/paper_debris/abnormal_analyte'
# Output folder for cropped images and mask images
output_folder = 'datasets/paper_debris/abnormal_output'

# Process each JSON file in the directory
for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        json_file = os.path.join(json_folder, filename)
        process_json(json_file, image_folder, output_folder, anomaly_folder)


Cropped image, mask, and mask_inverted for datasets/paper_debris/abnormal_analyte\2024-06-18 15-11-14.json have been generated and saved as 2024-06-18 15-11-14_cropped.png, 2024-06-18 15-11-14_mask.png, and 2024-06-18 15-11-14_mask_inverted.png
Anomaly mask cropped for datasets/paper_debris/abnormal_anomaly\2024-06-18 15-11-14.json has been generated and saved as 2024-06-18 15-11-14_anomaly_mask_crop.png
Anomaly mask cropped for datasets/paper_debris/abnormal_anomaly\2024-06-18 15-12-02.json has been generated and saved as 2024-06-18 15-12-02_anomaly_mask_crop.png
Anomaly mask cropped for datasets/paper_debris/abnormal_anomaly\2024-06-18 15-12-24.json has been generated and saved as 2024-06-18 15-12-24_anomaly_mask_crop.png
Anomaly mask cropped for datasets/paper_debris/abnormal_anomaly\2024-06-18 15-12-34.json has been generated and saved as 2024-06-18 15-12-34_anomaly_mask_crop.png
Anomaly mask cropped for datasets/paper_debris/abnormal_anomaly\2024-06-18 15-12-56.json has been gener

In [11]:
import os
import json
from PIL import Image, ImageDraw

# 定義函數來處理每個 JSON 檔案
def process_json(json_file, image_folder, output_folder, anomaly_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # 從 JSON 數據中獲取圖片文件名稱
    image_file = data['info']['name']
    image_path = os.path.join(image_folder, image_file)

    # 加載原始圖片
    original_image = Image.open(image_path)
    image_width, image_height = original_image.size

    # 創建空白的遮罩圖像（假定為黑色背景的RGB圖像）
    mask_image = Image.new('L', (image_width, image_height), color=0)
    draw = ImageDraw.Draw(mask_image)

    # 創建第三類型（遮罩內原始內容）的空白圖像
    mask_inverted_image = Image.new('RGB', (image_width, image_height), color=(0, 0, 0))

    # 定義要填充白色的類別
    categories_to_fill = ["hat"]

    # 變量以跟踪所有點的邊界框
    min_x, min_y = image_width, image_height
    max_x, max_y = 0, 0

    # 為每個指定的類別繪製遮罩
    for obj in data['objects']:
        category = obj['category']
        if category in categories_to_fill:
            segmentation = obj['segmentation']
            polygon = []
            for point in segmentation:
                polygon.append(tuple(point))
                # 更新邊界框
                min_x = min(min_x, point[0])
                min_y = min(min_y, point[1])
                max_x = max(max_x, point[0])
                max_y = max(max_y, point[1])
            draw.polygon(polygon, fill=255)  # 在遮罩中填充白色

    # 創建包含遮罩區域內原始內容的圖像
    mask_image_rgb = mask_image.convert("RGB")
    mask_inverted_image.paste(original_image, mask=mask_image)

    # 將原始圖片、遮罩圖片和反轉遮罩圖片裁剪到邊界框內
    cropped_image = original_image.crop((min_x, min_y, max_x, max_y))
    cropped_mask_image = mask_image.crop((min_x, min_y, max_x, max_y))
    cropped_mask_inverted_image = mask_inverted_image.crop((min_x, min_y, max_x, max_y))

    # 如果不存在，創建輸出文件夾
    os.makedirs(output_folder, exist_ok=True)

    # 將裁剪後的圖片保存到指定的文件夾中
    json_filename = os.path.splitext(os.path.basename(json_file))[0]
    cropped_image.save(os.path.join(output_folder, f'{json_filename}_cropped.png'))
    cropped_mask_image.save(os.path.join(output_folder, f'{json_filename}_mask.png'))
    cropped_mask_inverted_image.save(os.path.join(output_folder, f'{json_filename}_mask_inverted.png'))

    print(f'已生成並保存 {json_filename}_cropped.png、{json_filename}_mask.png 和 {json_filename}_mask_inverted.png')

    # 處理瑕疵部位資料夾以生成異常遮罩
    anomaly_json_folder = anomaly_folder  # 瑕疵部位 JSON 檔案所在的路徑

    # 處理瑕疵部位資料夾中的每個 JSON 檔案
    for anomaly_filename in os.listdir(anomaly_json_folder):
        if anomaly_filename.endswith('.json'):
            anomaly_json_file = os.path.join(anomaly_json_folder, anomaly_filename)

            with open(anomaly_json_file, 'r') as af:
                anomaly_data = json.load(af)

            # 創建空白的異常遮罩圖像（假定為黑色背景的RGB圖像）
            anomaly_mask_image = Image.new('L', (image_width, image_height), color=0)
            anomaly_draw = ImageDraw.Draw(anomaly_mask_image)

            # 為瑕疵部位 JSON 中的每個指定類別繪製遮罩
            for obj in anomaly_data['objects']:
                category = obj['category']
                if category in categories_to_fill:
                    segmentation = obj['segmentation']
                    polygon = []
                    for point in segmentation:
                        polygon.append(tuple(point))
                    anomaly_draw.polygon(polygon, fill=255)  # 在異常遮罩中填充白色

            # 將異常遮罩裁剪到邊界框內
            cropped_anomaly_mask_image = anomaly_mask_image.crop((min_x, min_y, max_x, max_y))

            # 將裁剪後的異常遮罩保存到指定的文件夾中
            anomaly_json_filename = os.path.splitext(os.path.basename(anomaly_json_file))[0]
            cropped_anomaly_mask_image.save(os.path.join(output_folder, f'{anomaly_json_filename}_anomaly_mask_crop.png'))

            print(f'已生成並保存 {anomaly_json_filename}_anomaly_mask_crop.png')

# 主要資料夾中包含正常圖片的 JSON 檔案路徑
json_folder = 'datasets/paper_debris/abnormal_analyte'
# 包含原始圖片的資料夾路徑
image_folder = 'datasets/paper_debris/abnormal_analyte'
# 用於裁剪圖片和遮罩圖片的輸出資料夾路徑
output_folder = 'datasets/paper_debris/abnormal_output'
# 包含瑕疵部位圖片的 JSON 檔案路徑
anomaly_folder = 'datasets/paper_debris/abnormal_anomaly'

# 處理主要資料夾中的每個 JSON 檔案
for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        json_file = os.path.join(json_folder, filename)
        process_json(json_file, image_folder, output_folder, anomaly_folder)


已生成並保存 2024-06-18 15-11-14_cropped.png、2024-06-18 15-11-14_mask.png 和 2024-06-18 15-11-14_mask_inverted.png
已生成並保存 2024-06-18 15-11-14_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-12-02_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-12-24_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-12-34_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-12-56_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-13-07_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-13-22_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-13-32_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-14-27_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-14-39_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-17-16_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-17-28_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-17-52_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-18-02_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-12-02_cropped.png、2024-06-18 15-12-02_mask.png 和 2024-06-18 15-12-02_mask_inverted.png
已生成並保存 2024-06-18 15-11-14_anomaly_mask_crop.png
已生成並保存 2024-06-18 15-12-02_anomaly_mask_crop.png


# 刪除多餘的檔案

In [ ]:
# 刪除多餘的檔案
import os

# Specify the directory to clean
directory = 'datasets/paper_debris/normal_crop_mask_inverted'

# Loop through each file in the directory
for filename in os.listdir(directory):
    # Check if the file does not end with '_mask_inverted.png'
    if not filename.endswith('_mask_inverted.png'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Delete the file
        os.remove(file_path)
        print(f'Removed {file_path}')

print('Clean up completed. Only _mask_inverted.png files are retained.')


Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-02_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-02_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-12_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-12_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-22_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-22_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-48_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-48_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-21-07_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-21-07_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-21-19_cropped.png
Removed datasets/paper_debris/normal_crop_mask_

# 重新命名圖檔

In [ ]:
import os

# 指定要重新命名的資料夾路徑
directory = 'datasets/paper_debris/mask_abnormal_anomaly'

# 獲取資料夾中的所有圖檔案
image_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

# 根據檔案名稱排序
image_files.sort()

# 確定要使用的命名位數
num_digits = len(str(len(image_files)))

# 遍歷每個圖檔案並重新命名
for idx, filename in enumerate(image_files):
    # 根據位數格式化新的檔案名稱
    new_filename = f"{idx:0{num_digits}d}.png"  # 如果是其他格式的圖片，這裡需要調整副檔名
    
    # 建立原始檔案的完整路徑
    old_filepath = os.path.join(directory, filename)
    new_filepath = os.path.join(directory, new_filename)
    
    # 重新命名檔案
    os.rename(old_filepath, new_filepath)
    print(f'Renamed {old_filepath} to {new_filepath}')

print('Rename process completed.')


Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-11-14_mask.png to datasets/paper_debris/mask_abnormal_anomaly\00.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-12-02_mask.png to datasets/paper_debris/mask_abnormal_anomaly\01.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-12-24_mask.png to datasets/paper_debris/mask_abnormal_anomaly\02.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-12-34_mask.png to datasets/paper_debris/mask_abnormal_anomaly\03.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-12-56_mask.png to datasets/paper_debris/mask_abnormal_anomaly\04.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-13-07_mask.png to datasets/paper_debris/mask_abnormal_anomaly\05.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-13-22_mask.png to datasets/paper_debris/mask_abnormal_anomaly\06.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\202

In [6]:
# 刪除多餘的檔案
import os

# Specify the directory to clean
directory = 'datasets/paper_debris/normal_crop_mask_inverted'

# Loop through each file in the directory
for filename in os.listdir(directory):
    # Check if the file does not end with '_mask_inverted.png'
    if not filename.endswith('_mask_inverted.png'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Delete the file
        os.remove(file_path)
        print(f'Removed {file_path}')

print('Clean up completed. Only _mask_inverted.png files are retained.')


Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-02_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-02_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-12_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-12_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-22_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-22_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-48_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-20-48_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-21-07_cropped.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-21-07_mask.png
Removed datasets/paper_debris/normal_crop_mask_inverted\2024-06-18 15-21-19_cropped.png
Removed datasets/paper_debris/normal_crop_mask_

# 重新命名圖檔

In [9]:
import os

# 指定要重新命名的資料夾路徑
directory = 'datasets/paper_debris/mask_abnormal_anomaly'

# 獲取資料夾中的所有圖檔案
image_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

# 根據檔案名稱排序
image_files.sort()

# 確定要使用的命名位數
num_digits = len(str(len(image_files)))

# 遍歷每個圖檔案並重新命名
for idx, filename in enumerate(image_files):
    # 根據位數格式化新的檔案名稱
    new_filename = f"{idx:0{num_digits}d}.png"  # 如果是其他格式的圖片，這裡需要調整副檔名
    
    # 建立原始檔案的完整路徑
    old_filepath = os.path.join(directory, filename)
    new_filepath = os.path.join(directory, new_filename)
    
    # 重新命名檔案
    os.rename(old_filepath, new_filepath)
    print(f'Renamed {old_filepath} to {new_filepath}')

print('Rename process completed.')


Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-11-14_mask.png to datasets/paper_debris/mask_abnormal_anomaly\00.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-12-02_mask.png to datasets/paper_debris/mask_abnormal_anomaly\01.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-12-24_mask.png to datasets/paper_debris/mask_abnormal_anomaly\02.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-12-34_mask.png to datasets/paper_debris/mask_abnormal_anomaly\03.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-12-56_mask.png to datasets/paper_debris/mask_abnormal_anomaly\04.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-13-07_mask.png to datasets/paper_debris/mask_abnormal_anomaly\05.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\2024-06-18 15-13-22_mask.png to datasets/paper_debris/mask_abnormal_anomaly\06.png
Renamed datasets/paper_debris/mask_abnormal_anomaly\202